In [1]:
from pyspark.sql import SparkSession

#  Spark 세션 생성
spark = SparkSession.builder.appName("241212_01_MLlib_clustering").getOrCreate()

24/12/13 13:40:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/13 13:40:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# 데이터 생성
data = [
    (0, 0, 4.0),  # user 0 rated item 0 with 4.0
    (0, 1, 2.0),
    (1, 1, 3.0),
    (1, 2, 1.0),
    (2, 0, 5.0),
    (2, 2, 4.0)
]
columns = ["user_id", "item_id", "rating"] # ckuster 

In [3]:
rating_df = spark.createDataFrame(data, columns)
rating_df.show()

+-------+-------+------+
|user_id|item_id|rating|
+-------+-------+------+
|      0|      0|   4.0|
|      0|      1|   2.0|
|      1|      1|   3.0|
|      1|      2|   1.0|
|      2|      0|   5.0|
|      2|      2|   4.0|
+-------+-------+------+



In [4]:
# user, item - rating 정보를 >> 그룹을 만든다. > 모델 > 예측 결과 - 그룹 

# 전처리 

In [ ]:
#피쳐선택, 전처리 >모델 학습   > 예측

In [10]:
user_item_matrix = rating_df.groupBy("user_id").pivot("item_id").avg("rating").fillna(0)  # 사용자와 아이템 사이의 매트릭스 

In [11]:
user_item_matrix.show() #클러스터링에 넣어주기 위한 전처리 

+-------+---+---+---+
|user_id|  0|  1|  2|
+-------+---+---+---+
|      0|4.0|2.0|0.0|
|      1|0.0|3.0|1.0|
|      2|5.0|0.0|4.0|
+-------+---+---+---+



# 피쳐 벡터

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler= VectorAssembler(inputCols= ["0", "1", "2"], outputCol="features") 

In [ ]:
assembler.transform(user_item_matrix)

In [17]:
user_features = assembler.transform(user_item_matrix)
user_features.show()

+-------+---+---+---+-------------+
|user_id|  0|  1|  2|     features|
+-------+---+---+---+-------------+
|      0|4.0|2.0|0.0|[4.0,2.0,0.0]|
|      1|0.0|3.0|1.0|[0.0,3.0,1.0]|
|      2|5.0|0.0|4.0|[5.0,0.0,4.0]|
+-------+---+---+---+-------------+



# 모델 생성 > 학습 

In [14]:
from pyspark.ml.clustering import KMeans

# 모델 생성 
kmeans= KMeans(k=2 , seed=1,featuresCol= "features", predictionCol="cluster" )
kmeans

KMeans_3d8c8836b904

In [18]:
#모델 학습 
model = kmeans.fit(user_features)
model

24/12/13 14:14:16 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/12/13 14:14:16 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


KMeansModel: uid=KMeans_3d8c8836b904, k=2, distanceMeasure=euclidean, numFeatures=3

In [19]:
# 모델을 이용한 예측 
clusters= model.transform(user_features)

In [20]:
# 유저의 클러스터링 결과 
clusters.show()

+-------+---+---+---+-------------+-------+
|user_id|  0|  1|  2|     features|cluster|
+-------+---+---+---+-------------+-------+
|      0|4.0|2.0|0.0|[4.0,2.0,0.0]|      0|
|      1|0.0|3.0|1.0|[0.0,3.0,1.0]|      0|
|      2|5.0|0.0|4.0|[5.0,0.0,4.0]|      1|
+-------+---+---+---+-------------+-------+



In [ ]:
# 사용자 그룹화 :유사한 취향의 사용자끼리 그룹으롤 묶어주는 것.
## 아이템 그룹화 : 아이템간의 군집화를 통해 사용자들에게 추천해줄 수 있음 .